In [135]:
import json

In [2]:
DOWN = "Do"
RIGHT = "R"
DIAG = "Di"
START = "S"

In [147]:
def load_blosum62(filepath="../data/blosum62.json"):
    with open(filepath, "r") as fin:
        return json.loads(fin.read())

In [148]:
blosum = load_blosum62()

In [149]:
blosum["A"]["A"]

4

In [150]:
def scoring_grid(v, w, blosum, sigma=5):    
    n = len(v)
    m = len(w)
    
    backtrack = []
    for i in range(n+1):
        backtrack.append([0]*(m+1))   
        
    for i in range(1,n+1):
        backtrack[i][0] = DOWN
    
    for j in range(1,m+1):
        backtrack[0][j] = RIGHT
    
    grid = []
    for i in range(n+1):
        grid.append([0]*(m+1))
        
    for i in range(n+1):
        grid[i][0] = -i*sigma
    for j in range(m+1):
        grid[0][j] = -j*sigma
        
    for i in range(1,n+1):
        for j in range(1,m+1):
            match = blosum[v[i-1]][w[j-1]]
            grid[i][j] = max(
                grid[i-1][j] - sigma,
                grid[i][j-1] - sigma,
                grid[i-1][j-1] + match 
            )
            if grid[i][j] == grid[i-1][j] - sigma:
                backtrack[i][j] = DOWN
            elif grid[i][j] == grid[i][j-1] - sigma:
                backtrack[i][j] = RIGHT
            elif grid[i][j] == grid[i-1][j-1] + match:
                backtrack[i][j] = DIAG
                
    return grid, backtrack

In [157]:
def traceback_path(backtrack, v, w, i, j):
    if j == 0 and i==0:
        return ("","")
    if backtrack[i][j] == DOWN:
        vv, ww = traceback_path(backtrack, v, w, i-1, j) 
        vv += v[i-1]
        ww += "-"
        return vv, ww
    elif backtrack[i][j] == RIGHT:
        vv, ww = traceback_path(backtrack, v, w, i, j-1) 
        vv += "-"
        ww += w[j-1]
        return vv, ww
    else:
        vv, ww = traceback_path(backtrack, v, w, i-1, j-1) 
        vv += v[i-1]
        ww += w[j-1]
        return vv, ww

In [160]:
def align(v, w):
    grid, backtrack =  scoring_grid(v, w, blosum)
    score = grid[len(grid)-1][len(grid[0])-1]
    aligned_v, aligned_w = traceback_path(backtrack, v, w, len(v), len(w))
    return score, aligned_v, aligned_w

In [161]:
def print_align(v, w):
    score, aligned_v, aligned_w = align(v, w)
    print(score)
    print(aligned_v)
    print(aligned_w)

In [162]:
print_align(v, w)

8
PLEASANTLY
-MEA--N-LY


In [163]:
v="ILYPRQSMICMSFCFWDMWKKDVPVVLMMFLERRQMQSVFSWLVTVKTDCGKGIYNHRKYLGLPTMTAGDWHWIKKQNDPHEWFQGRLETAWLHSTFLYWKYFECDAVKVCMDTFGLFGHCDWDQQIHTCTHENEPAIAFLDLYCRHSPMCDKLYPVWDMACQTCHFHHSWFCRNQEMWMKGDVDDWQWGYHYHTINSAQCNQWFKEICKDMGWDSVFPPRHNCQRHKKCMPALYAGIWMATDHACTFMVRLIYTENIAEWHQVYCYRSMNMFTCGNVCLRCKSWIFVKNYMMAPVVNDPMIEAFYKRCCILGKAWYDMWGICPVERKSHWEIYAKDLLSFESCCSQKKQNCYTDNWGLEYRLFFQSIQMNTDPHYCQTHVCWISAMFPIYSPFYTSGPKEFYMWLQARIDQNMHGHANHYVTSGNWDSVYTPEKRAGVFPVVVPVWYPPQMCNDYIKLTYECERFHVEGTFGCNRWDLGCRRYIIFQCPYCDTMKICYVDQWRSIKEGQFRMSGYPNHGYWFVHDDHTNEWCNQPVLAKFVRSKIVAICKKSQTVFHYAYTPGYNATWPQTNVCERMYGPHDNLLNNQQNVTFWWKMVPNCGMQILISCHNKMKWPTSHYVFMRLKCMHVLMQMEYLDHFTGPGEGDFCRNMQPYMHQDLHWEGSMRAILEYQAEHHRRAFRAELCAQYDQEIILWSGGWGVQDCGFHANYDGSLQVVSGEPCSMWCTTVMQYYADCWEKCMFA"
w="ILIPRQQMGCFPFPWHFDFCFWSAHHSLVVPLNPQMQTVFQNRGLDRVTVKTDCHDHRWKWIYNLGLPTMTAGDWHFIKKHVVRANNPHQWFQGRLTTAWLHSTFLYKKTEYCLVRHSNCCHCDWDQIIHTCAFIAFLDLYQRHWPMCDKLYCHFHHSWFCRNQEMSMDWNQWFPWDSVPRANCLEEGALIALYAGIWANSMKRDMKTDHACTVRLIYVCELHAWLKYCYTSINMLCGNVCLRCKSWIFVKLFYMYAPVVNTIEANSPHYYKRCCILGQGICPVERKSHCEIYAKDLLSFESCCSQKQNCYTDNWGLEYRLFFQHIQMECTDPHANRGWTSCQTAKYWHFNLDDRPPKEFYMWLQATPTDLCMYQHCLMFKIVKQNFRKQHGHANPAASTSGNWDSVYTPEKMAYKDWYVSHPPVDMRRNGSKMVPVWYPPGIWHWKQSYKLTYECFFTVPGRFHVEGTFGCNRWDHQPGTRRDRQANHQFQCPYSDTMAIWEHAYTYVDQWRSIKEGQMPMSGYPNHGQWNVHDDHTNEQERSPICNQPVLAKFVRSKNVSNHEICKKSQTVFHWACEAQTNVCERMLNNQHVAVKRNVTFWWQMVPNCLWSCHNKMTWPTRPEQHRLFFVKMRLKCMHEYLDVAPSDFCRNMQAYMHSMRAILEYQADFDLKRRLRAIAPMDLCAQYDQEIILWSGGYIYDQSLQVVSCEGCSYYADCYVKCINVKEKCMFA"

In [165]:
score, aligned_v, aligned_w = align(v, w)

In [166]:
expected_v = "ILYPRQSMICMSFCF-WDM--WKKDVPVVLMMFLERRQMQSVF-S-WL--VTVKTDCGKGIYNHR-K--Y-LGLPTMTAGDWHWIKK---Q-NDPHEWFQGRLETAWLHSTFLYWKYFE-CDAVKVCMDTFGLFGHCDWDQQIHTCTHENEPAIAFLDLYCRHSPMCDKLYPVWDMACQTCHFHHSWFCRNQEMWMKGDVDDWQWGYHYHTINSAQCNQWFKEICKDMGWDSVFPPRHNCQRHKKCMPALYAGIW---MA----TDHACTFMVRLIYTENIAEWHQVYCYRSMNMFTCGNVCLRCKSWIFVKN-YMMAPVVNDPMIEA--FYKRCCILGKAWYDMWGICPVERKSHWEIYAKDLLSFESCCSQKKQNCYTDNWGLEYRLFFQSIQMN-TDPH----Y--CQTHVCW-ISAMF-PIYSPFYT--SG-PKEFYMW---LQARI-DQNM---HGHANHYV-TSGNWDSVYTPEKRA--G--V-FP-V-V-------VPVWYPPQMCN--D-YIKLTYEC--E---RFHVEGTFGCNRWD-L-GCRR--YII--FQCPYCDTMKI---CY--VDQWRSIKEGQFRMSGYPNHGYWFVHDDHTNEW-----CNQPVLAKFVRSKIVA---ICKKSQTVFHYAYTPGYNATWPQTNVCERMYGPHDNLLNNQQNVTFWWKMVPNCGMQILISCHNKMKWPT--S-HYVF---MRLKCMHVLMQMEYLDHFTGPGEGDFCRNMQPYMHQDLHWEGSMRAILEYQAEHH-RRAFRA----ELCAQYDQEIILWSGGWGVQDCGFHANYDGSLQVVSGEPCSMWCTTVMQYYADCWEKCMFA"
expected_w = "ILIPRQQMGCFPFPWHFDFCFWSAHHSLVVP--LNP-QMQTVFQNRGLDRVTVKTDC----HDHRWKWIYNLGLPTMTAGDWHFIKKHVVRANNPHQWFQGRLTTAWLHSTFLY-KKTEYC-LVR---HS-NCC-HCDWDQIIHTCAF-----IAFLDLYQRHWPMCDKLY------C---HFHHSWFCRNQEMSM--D---W---------N--Q---WFP-------WDSV-P-RANCLE-EGALIALYAGIWANSMKRDMKTDHACT--VRLIYVCELHAWLK-YCYTSINML-CGNVCLRCKSWIFVKLFYMYAPVVNTIEANSPHYYKRCCILGQ------GICPVERKSHCEIYAKDLLSFESCCSQK-QNCYTDNWGLEYRLFFQHIQMECTDPHANRGWTSCQTAKYWHFNLDDRPP-KEFYMWLQATPTDLCMYQHCLMFKIVKQNFRKQHGHANPAASTSGNWDSVYTPEKMAYKDWYVSHPPVDMRRNGSKMVPVWYPPGIWHWKQSY-KLTYECFFTVPGRFHVEGTFGCNRWDHQPGTRRDRQANHQFQCPYSDTMAIWEHAYTYVDQWRSIKEGQMPMSGYPNHGQWNVHDDHTNEQERSPICNQPVLAKFVRSKNVSNHEICKKSQTVFHWA-C---EA---QTNVCERMLN-NQHVAV-KRNVTFWWQMVPNC----LWSCHNKMTWPTRPEQHRLFFVKMRLKCMH-----EYLD--VAPS--DFCRNMQAYMH-------SMRAILEYQADFDLKRRLRAIAPMDLCAQYDQEIILWSGGY-I--------YDQSLQVVSCEGCSYYADCYVKCI-NVKEKCMFA"

In [168]:
assert expected_v == aligned_v
assert expected_w == aligned_w

In [170]:
with open("../data/dataset_247_3.txt", "r") as fin:
    lines = [line.strip() for line in fin]
    v = lines[0]
    w = lines[1]
    print_align(v, w)

2379
MTSQNRFNNPQLDAPNLTNRYLLMDYLWVMANSHAQSDQIRIHMRTHEGMTNNLENYTQYAHRIILQFKCDVQMLFMMLEHETIAGEDRI-SWFKIRP-KYGDFHNEL-LTYTACEQCWKQVQQQRVITLPNPAVMRFPIFWC---A--------N--CYRAACEPKQSWDCMHQFYSMTQFAKKVGFHGKPQDMACGV--D-I-C-NQGYPHVGLNKANEAKKWFKHSMATCWPCWWP-SHDQIYPTRYKMVSSSENYCHIYWYKLTTKNFDFPDVIQHTKCCSDMCIIHQHRRYAIWRLDEACDHQLYEKLITCNQACDQPSIMATHVYFWKI--CFCHFIQLVPQIWY--LKYGQVKIS-------I-GLKSLMEIIQFNTQGK-N-TCINATPICFQTYAVMANTFSFTVCDAEWPHGSFGEFVVVTAVAQECSFKKVVFASVQWRPDADERYKSLRTGRTDKPKIKQVKTVA-FL----M-L-AVWYRQVVCHYLCFDSRKPRQP-LCMWSAWTHYMYCKRYVTDHSGW---CSWYGGFAQNREERRKCQKYVKMKGWLHWVVWKMRSEFHLMLNYLIEDAYEVEHWKHMNALNKGGRTCESRNEHRKRGNAIPKAMLDFHDPGWKWTAGINAIFDPWKCHLTQFLAGLCFYGQNCLMTSVSQKFHCINWYEAH--S--I-RWEVNHWVDFKLDCCGIVTKFMGKLMMEMVVRYTCMNNYVCEHG-PNRRPMGECDLSKYVTDSTYPMRKRHRQCNGDQSEAGNKEHMICSPCMRIHTWCIDKEERNGVPWMHMTGIDCRELI-GHSPTKGDNKHMFNFHVGLCQCRRHR-IYIDGRTVQMHRSHKTMSVIWPVKVEKVFGD--N-----YDWKLIPDQKLFFYRPCAVLNTQGERW-VQLRIN---IYPTKQPINGSYRVCWTRMV-T------QMDSVSPVKEVVMSNKYFSVAPFLSAP
MTSQNRFNNPQLDLPNLTNRYLLMDYL

In [1]:
#local alignment

In [1]:
with open("../data/PAM250.txt") as fin:
    lines = fin.read().split("\n")
    scoring_matrix = {l:dict() for l in lines[0].strip().split(" ") if l!=""}
    index2residue = {}
    c = 0
    for l in lines[0].strip().split(" "):
        if l!="":
            index2residue[c] = l
            c += 1
    for line in lines[1:]:
        ll = [l for l in line.strip().split(" ") if l!= ""]
        for i, v in enumerate(ll[1:]):
            scoring_matrix[ll[0]][index2residue[i]] = int(v)

In [30]:
def local_scoring_grid(v, w, scoring_matrix, sigma=5):    
    n = len(v)
    m = len(w)
    
    backtrack = []
    for i in range(n+1):
        backtrack.append([(0,0,"","")]*(m+1))   
        
    for i in range(1,n+1):
        backtrack[i][0] = (i-1,0,v[i-1],"-")
    
    for j in range(1,m+1):
        backtrack[0][j] = (0,j-1,"-",w[j-1])
    
    grid = []
    for i in range(n+1):
        grid.append([0]*(m+1))
        
    for i in range(n+1):
        grid[i][0] = -i*sigma
    for j in range(m+1):
        grid[0][j] = -j*sigma
        
    max_score = -1
    max_i = 0
    max_j = 0
    for i in range(1,n+1):
        for j in range(1,m+1):
            match = scoring_matrix[v[i-1]][w[j-1]]
            grid[i][j] = max(
                0,
                grid[i-1][j] - sigma,
                grid[i][j-1] - sigma,
                grid[i-1][j-1] + match 
            )
            if grid[i][j] > max_score:
                max_score = grid[i][j]
                max_i = i
                max_j = j
            if grid[i][j] == 0:
                backtrack[i][j] = (0,0,"","")
            elif grid[i][j] == grid[i-1][j] - sigma:
                backtrack[i][j] = (i-1,j,v[i-1],"-")
            elif grid[i][j] == grid[i][j-1] - sigma:
                backtrack[i][j] = (i,j-1,"-",w[j-1])
            elif grid[i][j] == grid[i-1][j-1] + match:
                backtrack[i][j] = (i-1,j-1,v[i-1],w[j-1])
                
    if max_i != n and max_j != m:
        backtrack[n][m] = (max_i, max_j, v[max_i-1], w[max_j-1])
        grid[n][m] = max_score
    
                
    return grid, backtrack

In [31]:
def local_traceback_path(backtrack, v, w, i, j):
    if j == 0 and i==0:
        return ("","")
    vv, ww = local_traceback_path(backtrack, v, w, backtrack[i][j][0], backtrack[i][j][1]) 
    vv += backtrack[i][j][2]
    ww += backtrack[i][j][3]
    return vv, ww

In [32]:
def local_align(v, w, scoring_matrix, sigma=5):
    grid, backtrack =  local_scoring_grid(v, w, scoring_matrix, sigma)
    score = grid[len(grid)-1][len(grid[0])-1]
    aligned_v, aligned_w = local_traceback_path(backtrack, v, w, len(v), len(w))
    return score, aligned_v, aligned_w

In [33]:
def print_local_align(v, w, scoring_matrix, sigma=5):
    score, aligned_v, aligned_w = local_align(v, w, scoring_matrix, sigma)
    print(score)
    print(aligned_v)
    print(aligned_w)

In [34]:
v = "MEANLY"
w = "PENALTY"

In [35]:
print_local_align(v,w, scoring_matrix, 5)

15
EANL-Y
ENALTY


In [36]:
with open("../data/dataset_247_10-2.txt", "r") as fin:
    lines = [line.strip() for line in fin]
    v = lines[0]
    w = lines[1]
    print_local_align(v, w, scoring_matrix, 5)

1373
FSRNY-PGVTETFNKDGRYLCLMPINQMEHP-LRPGQKRLPAIWRDDETEHVRPFNNVKPDPPM-CKCI-L-GTQTVQNK--MWH-DPIWSKWAVFNSQF-YPS-C-WFI-MVPFSWPNIC----AWKKVGLHIKTMGHMVYF-VPEAWFCGFSYTCNVQKWPGHELKPLHRNMREDMENYHECNTEHIPHLVTSKFNTT-KTDSG-AWYEVARQIKSLHMKLKQVHTTI-M-CYPCFYVDHHRRQTMICIDPKVFDSLHQQAWAFHCYHTLVQVMIFRYLLNA-A-PDH-QAMSAEWDNPWKTGDRHDYAHYAWKLGEKWNQAQFVAQ-F---IDDAW-CATDI----YKATRKDIVVMYHIPKLCDQHHIYSWNQRYDFGARFANPSWSITTSNIMPLFCLMLESPTHHF-RLAFHQRDFRCSYNM--------AHGSCEHDICFKIIPPPWLCIQSMAFINARP--PCNIGFEMIYHNHSWFGHEKWQFLTNACNWGYIGRLYKLQRCEKENCFQMIELFFGIKDQTTS-E-----I-N-C-G----G---GGKGSREHVM-LFM-E-------FLVVSRKENSV----EVMWQSCKRDAWCQGGFLGNP-W-K-FG--Q---PWRILMRVE--KP-T--FVMMGLEYEKTSNIKPFFNDSGESENIAWNEFFLAEINVL-QFV-MPCIYV-R---ARIHKVIEHGRLIP-EQDLNV--ARV-VARFVITTYQHLKWLHPGQ--Y-YVYPACDGRGKYCGYCRWLIS-CEP-KD-GT-EWKEYRAFTLCHQP-EWNAVP-RESFEQGSGIDVH-FIAVQN-KAGI-GMPRKQCIYE-ATIAH-ADQD-MIFN--F-IMTSPDVEDIDRFQQSMKM-KRIDVYISNAVF-QNYKAKDNVGICPVVMCWVPNNNSFVRSRTSSIQY-QCI-Q-FF-HRVYYSVWNA-LSINM-FPIKVKMMRIYMTAN-HTYY
YDFQYCDSKKQPI-KEGHDV

In [118]:
v="AMTAFRYRQGNPRYVKHFAYEIRLSHIWLLTQMPWEFVMGIKMPEDVFQHWRVYSVCTAEPMRSDETYEQKPKPMAKWSGMTIMYQAGIIRQPPRGDRGVSDRNYSQCGKQNQAQLDNNPTWTKYEIEWRVQILPPGAGVFEGDNGQNQCLCPNWAWEQPCQWGALHSNEQYPNRIHLWAPMSKLHIKIEKSSYNRNAQFPNRCMYECEFPSYREQVDSCHYENVQIAFTIFSGAEQKRKFCSCHFWSNFIDQAVFSTGLIPWCYRRDDHSAFFMPNWNKQYKHPQLQFRVAGEGTQCRPFYTREMFTKVSAWRIAGRFAGPYERHHDAHLELWYQHHKVRTGQQLGIIWNNRDKTRNPCPFSAYYNKLPWWKINQNAFYNCLQNIAHSTHDETHEFNPVKCIDWLQGTMVPTECKKGFVHEKCECYRNPGPPLHDMYHQMEDIFGVRFDCLTGWKHLSDYNPCQERRNINDFYIFAYEIAPAVKNLVLSPQPLADATKKCAFNYTPLDQSPVVIACKWYIHQPICMLLIVLICAMDKYNAHMIVIRTTEGQQPMHACRMTEGPGMCMKEPLVTFTLPAQWQWPNHEFKYVYMYVLNYHLSQYTYTDEGHAGGQHYSFNVAVDVGMAWGHNRCYCQPACYSQQETQTRTIDYEKWQYMKHQAFKWGLWFCEQERHAWFKGQNRCEMFTAKMTRMGADSNLDQYKLMLAQNYEEQWEQPIMECGMSEIIEIDPPYRSELIFTFWPFCTYSPWQNLIKCRCNNVIEEMDQCVPLTFIGFGVKQAGGIQAWAFYKEEWTSTYYLMCQCMKSDKAQYPYEIILFWMQPMDTGEQEPPQQNMWIFLPHSWFFDWCCNAPWSEICSSRHDHGQCQDAFYPCELFTVFDDIFTAEPVVCSCFYDDPM"
w="WQEKAVDGTVPSRHQYREKEDRQGNEIGKEFRRGPQVCEYSCNSHSCGWMPIFCIVCMSYVAFYCGLEYPMSRKTAKSQFIEWCDWFCFNHWTNWAPLSIVRTSVAFAVWGHCWYPCGGVCKTNRCKDDFCGRWRKALFAEGPRDWKCCKNDLQNWNPQYSQGTRNTKRMVATTNQTMIEWKQSHIFETWLFCHVIIEYNWSAFWMWMNRNEAFNSIIKSGYPKLLLTQYPLSQGSTPIVKPLIRRDQGKFWAWAQMWWFREPTNIPTADYCHSWWQSRADLQNDRDMGPEADASFYVEFWYWVRCAARTYGQQLGIIWNNRLKTRNPCPYSADGIQNKENYVFWWKNMCTKSHIAFYYCLQNVAHYTHDVTAEFNPVKCIDWLQGHMVLSSWFKYNTECKKLFVHEKCECYRMFCGVVEDIFGVRFHTGWKHLSTAKPVPHVCVYNPSVQERRNINDFYIFYEIAPAVKNLVLSAQPLHDYTKKCAFNYTPITITRIISTRNQIIWAHVVIACQFYSPHQMLLIELAMDKYCADMNVRRSTEGHQPMHACRSTFGPGMAAKEPLVTFTLVAFWQWPNHEFQYVYMYTEDKIIQIGPHLSNGCEMVEYCVDCYAKRPCYRAYSAEAQYWRMITEAEDYSYKTRNAIAATATVRGQYCHPFRWLGIVWMAHHDCFFANECGTICIPQMAEMRPPETTPYEIDIIFMMFWKEHMSTTILDVVGMYRPATFSHWHDAHHQCEPYLTPLMCQSKLVFDAAFTQVGVKGVWYHTEKLELMAGFNHMKFKKEEAQQSCFYWFQDCPDYDPPDAVRKTDEKHIRAHGEIWWLMRYYCMYHILHIASRHEWMHLRWDQACTNPGYELFEFIPWVLRRYVVYDKIRYNYSYRNSASMEFV"

In [119]:
score, vv, ww = local_align(v, w, scoring_matrix, 5)

In [109]:
vv=="YQAGIIRQPPRGD-RGVSDRNYSQCGKQ-NQ-AQLDNNPTWTKYEIEWRVQI-LPPGAGVFEGDNGQNQCLCPNW--A-W-EQPCQW----GALHS-NEQYPNRIHLWAPMSKLHIKIEKSSYN-RNAQ-FPNRCMYECE-FPSY-REQVDSCHYENVQIAF-TIFSGAEQKRKFCSCHFWSNFIDQAVFSTGLI-PWCYRRDDHSAFFMPNWNKQ--YKHPQLQFRVAGEGTQCRPFYTREMFTKVSAWRIAGRFAGPYERHHDAHLELWY-QHHKVRT-GQQLGIIWNNRDKTRNPCPFSAY-Y-NK--LP-WWK-I-NQ-N-AFYNCLQNIAHSTHDETHEFNPVKCIDWLQGTMV-P------TECKKGFVHEKCECYRNPGPPLHDMYHQMEDIFGVRFDCLTGWKHLS------D---YNPC-QERRNINDFYIFAYEIAPAVKNLVLSPQPLADATKKCAFNYTPLDQSPVVIACK---WYIHQPI-CMLL----IVLIC-AMDKYNAHMIVIRTTEGQQPMHACRMTEGPGMCMKEPLVTFTLPAQWQWPNHEFKYVYMYVLNYHLSQYTYTDEGHAGGQHYSFNVAVDVGMAWGHNRCYCQPACYSQQETQTRTIDYEKWQYMKHQAFKWGLWFCEQER-HA--WFKGQNRCEMFTAKMTRMGADSNLDQYKLMLAQNYEEQWEQPIMECGMSEIIEIDPPYRSELIFTFWPFCTYSPWQNLIKCRCNNVIEEMDQCVP-LTF-IGFGVKQAGGIQA-WAFYKE--EWTSTYYLMCQCMKSDKAQYPYEIILFWMQ--P-MDTGE--QEPPQQNMWIFLPHSWFFDWCCNAPWSEICSSRHD--H---GQ-CQDAFYPCELFTVF"

True

In [110]:
ww=="Y-P-MSRKTAKSQFIEWCDW-F--CFNHWTNWAPLSIVRTSVAFAV-W-GHCWYPCG-GVCKTNRCKDD-FCGRWRKALFAEGPRDWKCCKNDLQNWNPQYSQGTR--NTK-RMVATTNQTMIEWKQSHIFETW-LF-CHVIIEYNWSAF-W-MWMNRNEAFNSIIKSGYPKLLL-T-QY-P-L-SQG--STPIVKPL-IRRD-QGKFW-A-WAQMWWFREPT-NIPTA-D-Y-CHSW--WQ--SR-ADLQ-NDRDMGP-EADASFYVEFWYWVRCAARTYGQQLGIIWNNRLKTRNPCPYSADGIQNKENYVFWWKNMCTKSHIAFYYCLQNVAHYTHDVTAEFNPVKCIDWLQGHMVLSSWFKYNTECKKLFVHEKCECYRM----FCGV---VEDIFGVRFH--TGWKHLSTAKPVPHVCVYNPSVQERRNINDFYIF-YEIAPAVKNLVLSAQPLHDYTKKCAFNYTPITITRIISTRNQIIW-AHVVIACQFYSPHQMLLIELAMDKYCADMNVRRSTEGHQPMHACRSTFGPGMAAKEPLVTFTLVAFWQWPNHEFQYVYMYTED-KIIQIG-PHLSN-GCEMVEYCVDC-YAK-RPCYRAYSAEAQYWRMITEAEDYSYKTRNAIAATATVRGQ-YCHPFRWLGIVWM-AHHDC-FFANECGTICI-PQMAEMRPPETTPYEI--DIIFMMF-WKE--HMSTTIL-DVVGMYRP-ATFSHWHDAHH-QCEPYLTPL-MCQSKLVFDAAFT--QVG-VKGVW-YHTEKLELMAGFNHM-K-FKKEEAQ---QSCFYWFQDCPDYDPPDAVRKTDEKHIRAHGEIWWLMRYYCMYHILHI-ASRHEWMHLRWDQACTNPGY--ELFE-F"

True

In [120]:
score

1062